# 컨텍스추얼 압축
- 검색의 한 가지 문제는 데이터 저장 시스템에 데이터를 입력할 때 특정 쿼리를 알지 못한다는 점입니다.
- 이로 인해 쿼리와 가장 관련 있는 정보가 많은 불필요한 텍스트가 포함된 문서에 묻힐 수 있습니다.
- 전체 문서를 애플리케이션에 전달하면 더 비싼 LLM 호출과 더 나쁜 응답을 초래할 수 있습니다.
 
# 컨텍스추얼 압축은 이를 해결하기 위한 것입니다.
- 아이디어는 간단합니다: 검색된 문서를 그대로 반환하는 대신 주어진 쿼리의 컨텍스트를 사용하여 압축할 수 있습니다.
- 여기서 "압축"은 개별 문서의 내용을 압축하고 문서를 전체적으로 필터링하는 것을 의미합니다.
 
# 컨텍스추얼 압축 검색기를 사용하려면 다음이 필요합니다:
- 기본 검색기
- 문서 압축기
- 컨텍스추얼 압축 검색기는 쿼리를 기본 검색기에 전달하고 초기 문서를 받아 문서 압축기에 전달합니다.
- 문서 압축기는 문서 목록을 받아 문서의 내용을 줄이거나 문서를 완전히 삭제하여 목록을 단축합니다.

In [1]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

 # 기본 벡터 스토어 검색기 사용
- 간단한 벡터 스토어 검색기를 초기화하고 2023년 국정 연설을 (청크로) 저장해보겠습니다.
- 예시 질문을 주면 검색기가 한두 개의 관련 문서와 몇 개의 관련 없는 문서를 반환하는 것을 볼 수 있습니다.
- 그리고 관련 문서에도 많은 관련 없는 정보가 포함되어 있습니다.


In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv
load_dotenv('../dot.env')

documents = TextLoader("./files/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.invoke("What did the president say about Ketanji Brown Jackson")
pretty_print_docs(docs)

Document 1:

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

A former top litigator in private practice. A former federal public defender. And fro

- 컨텍스추얼 압축을 LLMChainExtractor와 함께 추가
- 기본 검색기를 ContextualCompressionRetriever로 감쌉니다.
- LLMChainExtractor를 추가하여 초기 반환된 문서를 반복하고 각 문서에서 쿼리와 관련된 내용만 추출합니다.

In [4]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson.


- 더 많은 내장 압축기: 필터
- LLMChainFilter
- LLMChainFilter는 문서 내용을 조작하지 않고 처음 검색된 문서 중 어떤 것을 필터링하고 반환할지 결정하는 LLM 체인을 사용하는 더 간단하지만 더 강력한 압축기입니다.

In [6]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


- EmbeddingsFilter
- 각 검색된 문서에 대해 추가 LLM 호출을 하는 것은 비용이 많이 들고 느립니다.
- EmbeddingsFilter는 문서와 쿼리를 임베딩하여 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.


In [7]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

A former top litigator in private practice. A former federal public defender. And fro

- 압축기와 문서 변환기를 함께 연결하기
- DocumentCompressorPipeline을 사용하여 여러 압축기를 순차적으로 쉽게 결합할 수 있습니다.
- 압축기와 함께 BaseDocumentTransformers를 파이프라인에 추가할 수 있습니다. 이들은 문서 집합에 대해 단순히 변환을 수행하며, 맥락 압축은 수행하지 않습니다.
- 예를 들어 TextSplitters는 문서를 더 작은 조각으로 나누기 위해 문서 변환기로 사용할 수 있으며, EmbeddingsRedundantFilter는 문서 간 임베딩 유사성을 기반으로 중복 문서를 필터링하는 데 사용할 수 있습니다.
- 아래에서는 문서를 더 작은 청크로 나눈 다음 중복 문서를 제거하고 쿼리와의 관련성을 기준으로 필터링하여 압축기 파이프라인을 만듭니다.

In [8]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [9]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson
----------------------------------------------------------------------------------------------------
Document 2:

As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year
